# This should take
- an epitope list (consisting of ID, amino acid sequence) of length _n_
- a _k_ to generate
- an inter-epitope linker (amino acid, hard code for now)
- a scaffold-epitope linker (amino acid; hard code)
- account for specified ORF?

## some math stuff
- _n_ choose _k_ without replacement
# output
- a list of epitope_IDS (i.e. ID-ID-ID), Xmas insert to be cloned into GFP -- the gBlock, basically
- the final GFP-epitope structure for each
- check that final structure does not exceed AAV payload (section between ITRs)
## bonus items
- codon optimization
- a snapgene/DNA scaffold
- restriction cloning mimc

# some biology stuff, i.e. order of events
For TBD reasons, DNAchisel doesn't work on OsX for conda istall (with no visible errors). Probably a version issue, but ?Workflow is:
1. AA: epitope combos + appropriate linkers
2. codon-optimize (mus musculus, avoid EcoRI + BsrGI sites) to DNA seq
3. Add cloning tails (known; identical) with primer length?
    primers can come from main GFP
4. Output gBlock sequences (how does Genscript want these?)


In [1]:
import itertools
import pandas as pd
from itertools import combinations
import random

In [2]:
def rSubset(arr, r):
    '''return list of all subsets from an array of length r - i.e. from array choose k'''
    # return list of all subsets of length r
    # to deal with duplicate subsets use 
    # set(list(combinations(arr, r)))
    return list(combinations(arr, r))

In [3]:
def triplicator(epitope_sets, repeats=1):
    '''takes a list of single epitopes and generates aa amino acid with # linkrs'''
    triple_set = []
    for ep in epitope_sets:
        new_ep = [(ep) for _ in range(3)]
        triple_set.append(new_ep)
    return triple_set

In [43]:
def makeTag(epitope_sets, linker=None, prepend_linker=None, append_linker=None):
    '''from a list of tuples, combine name sets + epitope sets with specified inter-epitope.
    just some string manip, basically.
    shuffles multi-epitope n choose k setups
    if a prepend is supplied, will add the specified prepend'''
    # collector of new things
    knockin = [] #the tag we're knocking in
    
    # loop the old things
    for epitope_set in epitope_sets:
        # construct name and seq into new lists
        name = []
        seq = []
        
        #shuffler to avoid bias in epitope success
        to_shuffle = list(epitope_set)
        random.shuffle(to_shuffle) 
        epitope_set = tuple(to_shuffle)
        
        # make array
        for i in range(len(epitope_set)):
            name.append(epitope_set[i][0].replace(' ',''))
            seq.append(epitope_set[i][1].strip(' '))

        # construct final name and sequence and append
        name = f'_'.join(name) # name that puppy
        seq = f'{linker}'.join(seq) #make the tag sequence
        
        if prepend_linker:
            seq = prepend_linker + seq
        if append_linker:
            seq = seq + append_linker
        knockin.append((name, seq)) #add to array

    # return new things as a dictionary
    return dict(knockin)

In [44]:
interEp = "GGSGGS" #[GGS]2. for in between epitopes
interScaffEp = "GGSGGS" #linker between eGFP end and beginning of tag

preEpitopee ="ggacgagctgtacaag" #end of eGFP with BsrGI site
endEpitopes = "taagaattcgatatcaag" #stop codon + EcoRI


In [45]:
def read_list(csv, columns = None):
    '''returns records of name + aa seq'''
    epitopes = pd.read_csv(csv)
    records = epitopes[columns].to_records(index=False)
    return records

In [46]:
xmas_list = read_list('xmas-in-july_tagSet.csv', columns = ['Name', 'AA seq'])
all_list = read_list('xmas-in-july_tagSet.csv', columns = ['Name', 'AA seq'])
jun_list = read_list('Human Protein Atlas - under 40 AAs.csv', columns = ['Antibody','Antigen Sequence'])

In [47]:
xmas_sets = rSubset(xmas_list, 3)
singlets = rSubset(all_list,1)
triplicates = triplicator(all_list,3)
jun_triplicates = triplicator(jun_list,3)

In [40]:
# knockin_list = [xmas_sets, singles, triplicates, jun_triplicates]
# for item in knockin_list:
    

In [51]:
xmas_Tags = makeTag(xmas_sets, interEp,"GGSGGS") #make amino acid combinations
singlet_Tags = makeTag(singlets, interEp,"GGSGGS") #make amino acid combinations
triplet_Tags = makeTag(triplicates, interEp, "GGSGGS")
jun_Tags = makeTag(jun_triplicates, interEp, "GGSGGS")

# notes
x shuffle to avoid biasing
- add additional tags not covered in july
x jun's 